### Imports

In [44]:
import pandas as pd
import numpy as np
import sklearn
import pickle

### Loading Dataset

In [45]:
data_train = pd.read_csv('train.csv')


In [46]:
data_test=pd.read_csv('test.csv')
sub=pd.read_csv('gender_submission.csv')

In [47]:
data_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S
115,116,0,3,"Pekoniemi, Mr. Edvard",male,21.00,0,0,STON/O 2. 3101294,7.9250,NaN,S
138,139,0,3,"Osen, Mr. Olaf Elon",male,16.00,0,0,7534,9.2167,NaN,S
547,548,1,2,"Padro y Manent, Mr. Julian",male,NaN,0,0,SC/PARIS 2146,13.8625,NaN,C
397,398,0,2,"McKane, Mr. Peter David",male,46.00,0,0,28403,26.0000,NaN,S
250,251,0,3,"Reed, Mr. James George",male,NaN,0,0,362316,7.2500,NaN,S
486,487,1,1,"Hoyt, Mrs. Frederick Maxfield (Jane Anne Forby)",female,35.00,1,0,19943,90.0000,C93,S
132,133,0,3,"Robins, Mrs. Alexander A (Grace Charity Laury)",female,47.00,1,0,A/5. 3337,14.5000,NaN,S
372,373,0,3,"Beavan, Mr. William Thomas",male,19.00,0,0,323951,8.0500,NaN,S
675,676,0,3,"Edvardsson, Mr. Gustaf Hjalmar",male,18.00,0,0,349912,7.7750,NaN,S


In [48]:
column_names = data_train.columns
for column in column_names:
    print(column + ' - ' + str(data_train[column].isnull().sum()))

PassengerId - 0
Survived - 0
Pclass - 0
Name - 0
Sex - 0
Age - 177
SibSp - 0
Parch - 0
Ticket - 0
Fare - 0
Cabin - 687
Embarked - 2


### Pre-processing Data

In [49]:
# def pre_process(dat):
#     data=dat
#     data['Age'].fillna(data['Age'].mean(), inplace = True)
#     data['Fare'].fillna(data['Fare'].mean(), inplace = True)
#     data['Embarked'].fillna(data['Embarked'].bfill(), inplace = True)
#     data['Sex'] = data['Sex'].map({'male':0, 'female':1})
#     data['Embarked'] = data['Embarked'].map({'C':0, 'Q':1, 'S':2})
#     test= data.drop([ 'Name','Ticket', 'Cabin'], axis = 1)
#     return test

In [50]:
data_train['Age'].fillna(data_train['Age'].mean(), inplace = True)
data_train['Fare'].fillna(data_train['Fare'].mean(), inplace = True)
data_train['Embarked'].fillna(data_train['Embarked'].bfill(), inplace = True)

data_test['Age'].fillna(data_test['Age'].mean(), inplace = True)
data_test['Fare'].fillna(data_test['Fare'].mean(), inplace = True)
data_test['Embarked'].fillna(data_test['Embarked'].bfill(), inplace = True)

In [51]:
column_names = data_train.columns
for column in column_names:
    print(column + ' - ' + str(data_train[column].isnull().sum()))

PassengerId - 0
Survived - 0
Pclass - 0
Name - 0
Sex - 0
Age - 0
SibSp - 0
Parch - 0
Ticket - 0
Fare - 0
Cabin - 687
Embarked - 0


In [52]:
data_train['Sex'] = data_train['Sex'].map({'male':0, 'female':1})
data_train['Embarked'] = data_train['Embarked'].map({'C':0, 'Q':1, 'S':2})
data_test['Sex'] = data_test['Sex'].map({'male':0, 'female':1})
data_test['Embarked'] = data_test['Embarked'].map({'C':0, 'Q':1, 'S':2})

In [53]:
passenger_test=pd.DataFrame(data_test['PassengerId'])
y_train=data_train['Survived']
y_test=sub['Survived']
x_train = data_train.drop(['PassengerId','Survived', 'Name', 'Ticket', 'Cabin'], axis = 1)
x_test= data_test.drop([ 'PassengerId','Name','Ticket', 'Cabin'], axis = 1)
x_train.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
448,3,1,5.000000,2,1,19.2583,0
867,1,0,31.000000,0,0,50.4958,2
86,3,0,16.000000,1,3,34.3750,2
518,2,1,36.000000,1,0,26.0000,2
614,3,0,35.000000,0,0,8.0500,2
816,3,1,23.000000,0,0,7.9250,2
257,1,1,30.000000,0,0,86.5000,2
214,3,0,29.699118,1,0,7.7500,1
434,1,0,50.000000,1,0,55.9000,2
864,2,0,24.000000,0,0,13.0000,2


In [54]:
x_test.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
87,3,1,18.00000,0,0,8.0500,2
381,3,0,26.00000,0,0,7.8792,1
167,3,0,18.00000,2,2,34.3750,2
410,3,1,30.27259,0,0,7.7500,1
362,2,1,31.00000,0,0,21.0000,2
160,3,1,30.27259,0,0,7.7333,1
205,1,0,30.27259,0,0,25.7417,0
328,2,0,29.00000,1,0,26.0000,2
226,3,0,23.00000,0,0,7.7958,2
238,2,1,18.00000,1,1,13.0000,2


In [55]:
# def remove_pk(data):
#     new_data=data.loc[ : , data.columns != 'PassengerId']
#     return new_data

### Creating Decision tree Model

In [56]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_depth = 10, min_samples_split = 20)
decision_tree.fit(x_train,y_train)
y_pred = decision_tree.predict(x_test)

In [57]:
class DecisionTreeModel:
    def __init__(self,max_depth=10,min_samples_split=20):
        self.model = DecisionTreeClassifier(max_depth=max_depth,min_samples_split=min_samples_split)
    def pre(self,dat):
        data=dat.copy(deep=True)
        data['Age'].fillna(data['Age'].mean(), inplace = True)
        data['Fare'].fillna(data['Fare'].mean(), inplace = True)
        data['Embarked'].fillna(data['Embarked'].bfill(), inplace = True)
        data['Sex'] = data['Sex'].map({'male':0, 'female':1})
        data['Embarked'] = data['Embarked'].map({'C':0, 'Q':1, 'S':2})
        test= data.drop([ 'PassengerId','Name','Ticket', 'Cabin'], axis = 1)
        return test
    def fit(self,x,y):
        self.model.fit(x,y)
    def predict(self,data):
        processed=self.pre(data)
        return self.model.predict(processed)
        
    

In [58]:
# decision_tree = DecisionTreeModel()
# decision_tree.fit(remove_pk(x_train),y_train)

In [59]:
y_pred = decision_tree.predict(x_test)

In [60]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE : ',RMSE)

cm = confusion_matrix(y_test, y_pred)
print(cm)

RMSE :  0.39735970711951313
[[249  17]
 [ 49 103]]


In [61]:
#query_data=pd.concat([x_train, x_test])
#query_data

### Saving the model and pre-processed test data set

In [62]:
pickle_out = open("model.pickle","wb")
#query_data.to_csv('testing_data.csv')
#passenger_test.to_csv('passenger.csv',index=False)
pickle.dump(decision_tree, pickle_out)
pickle_out.close()

### Loading it to check validity

In [63]:
pickle_in = open("model.pickle","rb")
dt2 = pickle.load(pickle_in)
pickle_in.close()
#testing_data=pd.read_csv('testing_data.csv',index_col=[0])

In [64]:
dt2.predict(x_test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,